In [28]:
import pandas as pd
import numpy as np
import matplotlib as mp
import matplotlib.pyplot as plt
import scipy
import sklearn
from io import StringIO
import sys
%matplotlib inline

In [29]:
csv_data = \
'''A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
10.0,11.0,12.0,'''

df=pd.read_csv(StringIO(csv_data))
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [30]:
df.isnull().sum()

A    0
B    0
C    1
D    1
dtype: int64

In [31]:
df.values

array([[ 1.,  2.,  3.,  4.],
       [ 5.,  6., nan,  8.],
       [10., 11., 12., nan]])

In [32]:
df.dropna(axis=0)

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [33]:
df.values

array([[ 1.,  2.,  3.,  4.],
       [ 5.,  6., nan,  8.],
       [10., 11., 12., nan]])

In [34]:
df.dropna(axis=1)

,A,B
0,1.0,2.0
1,5.0,6.0
2,10.0,11.0


In [35]:
df.dropna(how='all')

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [36]:
df.dropna(subset=['C'])

,A,B,C,D
0,1.0,2.0,3.0,4.0
2,10.0,11.0,12.0,NaN


In [40]:
from sklearn.impute import SimpleImputer
imr = SimpleImputer(missing_values=np.nan, strategy ='mean')
imr = imr.fit(df.values)
imputed_data = imr.transform(df.values)
imputed_data


array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [10. , 11. , 12. ,  6. ]])

In [41]:
df.fillna(df.mean())

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,7.5,8.0
2,10.0,11.0,12.0,6.0


In [43]:
df.values

array([[ 1.,  2.,  3.,  4.],
       [ 5.,  6., nan,  8.],
       [10., 11., 12., nan]])

In [57]:
df = pd.DataFrame([['green','M',10.1,'class2'],
                  ['red','L',13.5,'class1'],
                  ['blue','XL',15.3,'class2']])
df.columns = ['color','size','price','classlabel']
df

,color,size,price,classlabel
0,green,M,10.1,class2
1,red,L,13.5,class1
2,blue,XL,15.3,class2


In [58]:
size_mapping={'XL':3,
             'L':2,
             'M':1}
df['size']= df['size'].map(size_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class2
1,red,2,13.5,class1
2,blue,3,15.3,class2


In [59]:
inv_size_mapping = {v: k for k, v in size_mapping.items()}
df['size'].map(inv_size_mapping)

0     M
1     L
2    XL
Name: size, dtype: object

In [60]:
class_mapping = {label : idx for idx, label in enumerate(np.unique(df['classlabel']))}
class_mapping

{'class1': 0, 'class2': 1}

In [61]:
df['classlabel'] = df['classlabel'].map(class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,1
1,red,2,13.5,0
2,blue,3,15.3,1


In [62]:
inv_class_mapping = {v: k for k, v in class_mapping.items()}
df['classlabel'] = df['classlabel'].map(inv_class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class2
1,red,2,13.5,class1
2,blue,3,15.3,class2


In [67]:
from sklearn.preprocessing import LabelEncoder

class_le= LabelEncoder()
y = class_le.fit_transform(df['classlabel'].values)
y

array([1, 0, 1])

In [71]:
class_le.inverse_transform(y)

array(['class2', 'class1', 'class2'], dtype=object)

In [78]:
x = df[['color','size','price',]].values
color_le=LabelEncoder()
x[:,0] = color_le.fit_transform(x[:,0])
x

array([[1, 1, 10.1],
       [2, 2, 13.5],
       [0, 3, 15.3]], dtype=object)

In [80]:
from sklearn.preprocessing import OneHotEncoder

x =df[['color','size','price']].values
color_ohe= OneHotEncoder()
color_ohe.fit_transform(x[:,0].reshape(-1,1)).toarray()

array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

In [89]:
from sklearn.compose import ColumnTransformer

x = df[['color','size','price']].values
c_transf=ColumnTransformer([('onehot',color_ohe,[0]),
                           ('nothing','passthrough',[1,2])])
c_transf.fit_transform(x).astype(float)

array([[ 1. ,  0. ,  1. , 10.1],
       [ 0. ,  1. ,  2. , 13.5],
       [ 0. ,  0. ,  3. , 15.3]])

In [90]:
pd.get_dummies(df[['price','color','size']])

,price,size,color_blue,color_green,color_red
0,10.1,1,0,1,0
1,13.5,2,0,0,1
2,15.3,3,1,0,0


In [91]:
pd.get_dummies(df[['price','color','size']], drop_first=True)

,price,size,color_green,color_red
0,10.1,1,1,0
1,13.5,2,0,1
2,15.3,3,0,0


In [92]:
color_ohe = OneHotEncoder(categories = 'auto',drop='first')
c_transf=ColumnTransformer([('onehot',color_ohe,[0]),
                           ('nothing','passthrough',[1,2])])
c_transf.fit_transform(x).astype(float)

array([[ 1. ,  0. ,  1. , 10.1],
       [ 0. ,  1. ,  2. , 13.5],
       [ 0. ,  0. ,  3. , 15.3]])

In [95]:
df_wine = pd.read_csv('https://archive.ics.uci.edu/'
                      'ml/machine-learning-databases/wine/wine.data',
                     header=None)
df_wine.columns=['Class label','Alcohol','Malic acid','Ash',
                'Alcalinity of ash','Magnesium','Total phenols',
                'Flavonoids','Nonflavonoid phenols','Proanthocyanins',
                'Color intensity','Hue','OD280/OD315 of diluted wines',
                'Proline']
print('Class labels',np.unique(df_wine['Class label']))
df_wine.head()

Class labels [1 2 3]


,Class label,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavonoids,Nonflavonoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [98]:
from sklearn.model_selection import train_test_split

X,y = df_wine.iloc[:,1:].values,df_wine.iloc[:,0].values

X_train,X_test,y_train,y_test =\
    train_test_split(X,y,
                    test_size=0.3,
                    random_state=0,
                    stratify=y)

In [99]:
from sklearn.preprocessing import MinMaxScaler

mms = MinMaxScaler()
X_train_norm=mms.fit_transform(X_train)
X_test_norm = mms.transform(X_test)

In [100]:
from sklearn.preprocessing import StandardScaler

stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.transform(X_test)

In [101]:
ex = np.array([0,1,2,3,4,5])
print('Standardized:',(ex-ex.mean())/ex.std())
print('Normalized : ',(ex-ex.min())/ (ex.max()- ex.min()))

Standardized: [-1.46385011 -0.87831007 -0.29277002  0.29277002  0.87831007  1.46385011]
Normalized :  [0.  0.2 0.4 0.6 0.8 1. ]
